In [1]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as stats
import pandas.plotting as pd_plot

import missingno as msno
import pandas_profiling

import re

import statsmodels.imputation.mice

from sklearn.impute import KNNImputer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import svm
from sklearn import metrics
from sklearn.preprocessing  import StandardScaler 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.preprocessing import scale
from sklearn.linear_model import LassoCV
import random

# Feature Selection/Dimension Reduction

In [ ]:
OriginalDatasetwTypes = pd.read_csv("CleanedDataset.csv")
OriginalDataset = OriginalDatasetwTypes.copy().drop(["Unnamed: 0", "Sleeves", "P&P", "Frac Ports", \
                                      "Repeater Ports", "CT"], axis = 1)
DatasetDropna = pd.read_csv("NoNanDataset.csv")
DatasetMean = pd.read_csv("MeanImputationDS.csv")
DatasetII = pd.read_csv("IterativeImputeDS.csv")
imputer = KNNImputer(n_neighbors=10)
DatasetKNN = imputer.fit_transform(OriginalDataset.copy().select_dtypes([np.number]))
DatasetKNN = pd.DataFrame(DatasetKNN, columns = OriginalDataset.copy().select_dtypes([np.number]).columns)
from impyute.imputation import cs
np.random.seed(9)
data = OriginalDataset.copy().select_dtypes([np.number]).to_numpy()
DatasetMICE = cs.mice(data)
DatasetMICE = pd.DataFrame(DatasetMICE, columns = OriginalDataset.copy().select_dtypes([np.number]).columns)
DatasetIIandCI = pd.read_csv("DatasetIIandCI.csv")

## LASSO Regression

### Dataset with Dropped Data

In [ ]:
from math import sqrt
X = DatasetDropna.copy().select_dtypes([np.number]).drop(["12 month Cum Prod", "Year Drilled"], axis = 1)
scaler = StandardScaler()
sX = scaler.fit_transform(X)
y = DatasetDropna["12 month Cum Prod"]
sy = scaler.fit_transform(y.values.reshape((-1,1)))
alphas = 10**np.linspace(10,-2, 100)*0.05
coefs = []
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .4, random_state = 1)
Lasso = linear_model.Lasso(max_iter = 300000, tol = 0.0001, normalize = True)
for a in alphas:
    Lasso.set_params(alpha = a)
    Lasso.fit(X_train, y_train)
    coefs.append(Lasso.coef_)
ax = plt.gca()
ax.plot(alphas*2, coefs)
ax.set_xscale("log")
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('weights')
    
Lassocv = LassoCV(alphas = None, cv = 20, max_iter = 300000, normalize = True)
Lassocv.fit(X_train, y_train)

Lasso.set_params(alpha = Lassocv.alpha_)
Lasso.fit(X_train, y_train)

y_pred = Lasso.predict(X_test)
rmse = sqrt(metrics.mean_squared_error(y_test, y_pred))

print("Train score:", Lasso.score(X_train, y_train))
print("Test score:", Lasso.score(X_test, y_test))
print("Number of coefficients used:", np.sum(Lasso.coef_!=0))
print("Root mean squared error:", rmse)
print(Lasso.coef_)
pd.Series(Lasso.coef_, index = X.columns)

#### Analysis

### Mean Imputation Dataset

In [ ]:
X = DatasetMean.copy().select_dtypes([np.number]).drop(["12 month Cum Prod", "Year Drilled"], axis = 1)
scaler = StandardScaler()
sX = scaler.fit_transform(X)
y = DatasetMean["12 month Cum Prod"]
sy = scaler.fit_transform(y.values.reshape((-1,1)))
alphas = 10**np.linspace(10,-2, 100)*0.05
coefs = []
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .4, random_state = 1)
Lasso = linear_model.Lasso(max_iter = 300000, tol = 0.0001, normalize = True)
for a in alphas:
    Lasso.set_params(alpha = a)
    Lasso.fit(X_train, y_train)
    coefs.append(Lasso.coef_)
ax = plt.gca()
ax.plot(alphas*2, coefs)
ax.set_xscale("log")
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('weights')
    
Lassocv = LassoCV(alphas = None, cv = 20, max_iter = 300000, normalize = True)
Lassocv.fit(X_train, y_train)

Lasso.set_params(alpha = Lassocv.alpha_)
Lasso.fit(X_train, y_train)

y_pred = Lasso.predict(X_test)
rmse = sqrt(metrics.mean_squared_error(y_test, y_pred))

print("Train score:", Lasso.score(X_train, y_train))
print("Test score:", Lasso.score(X_test, y_test))
print("Number of coefficients used:", np.sum(Lasso.coef_!=0))
print("Root mean squared error:", rmse)
print(Lasso.coef_)
pd.Series(Lasso.coef_, index = X.columns)

#### Analysis

### Iterative Imputatoin Dataset

In [ ]:
X = DatasetII.copy().select_dtypes([np.number]).drop(["12 month Cum Prod", "Year Drilled"], axis = 1)
scaler = StandardScaler()
sX = scaler.fit_transform(X)
y = DatasetII["12 month Cum Prod"]
sy = scaler.fit_transform(y.values.reshape((-1,1)))
alphas = 10**np.linspace(10,-2, 100)*0.05
coefs = []
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .4, random_state = 1)
Lasso = linear_model.Lasso(max_iter = 300000, tol = 0.0001, normalize = True)
for a in alphas:
    Lasso.set_params(alpha = a)
    Lasso.fit(X_train, y_train)
    coefs.append(Lasso.coef_)
ax = plt.gca()
ax.plot(alphas*2, coefs)
ax.set_xscale("log")
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('weights')
    
Lassocv = LassoCV(alphas = None, cv = 20, max_iter = 300000, normalize = True)
Lassocv.fit(X_train, y_train)

Lasso.set_params(alpha = Lassocv.alpha_)
Lasso.fit(X_train, y_train)

y_pred = Lasso.predict(X_test)
rmse = sqrt(metrics.mean_squared_error(y_test, y_pred))

print("Train score:", Lasso.score(X_train, y_train))
print("Test score:", Lasso.score(X_test, y_test))
print("Number of coefficients used:", np.sum(Lasso.coef_!=0))
print("Root mean squared error:", rmse)
print(Lasso.coef_)
pd.Series(Lasso.coef_, index = X.columns)

#### Analysis

### K- NN Imputation Dataset

In [ ]:
X = DatasetKNN.copy().select_dtypes([np.number]).drop(["12 month Cum Prod", "Year Drilled"], axis = 1)
scaler = StandardScaler()
sX = scaler.fit_transform(X)
y = DatasetKNN["12 month Cum Prod"]
sy = scaler.fit_transform(y.values.reshape((-1,1)))
alphas = 10**np.linspace(10,-2, 100)*0.05
coefs = []
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .4, random_state = 1)
Lasso = linear_model.Lasso(max_iter = 300000, tol = 0.0001, normalize = True)
for a in alphas:
    Lasso.set_params(alpha = a)
    Lasso.fit(X_train, y_train)
    coefs.append(Lasso.coef_)
ax = plt.gca()
ax.plot(alphas*2, coefs)
ax.set_xscale("log")
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('weights')
    
Lassocv = LassoCV(alphas = None, cv = 20, max_iter = 300000, normalize = True)
Lassocv.fit(X_train, y_train)

Lasso.set_params(alpha = Lassocv.alpha_)
Lasso.fit(X_train, y_train)

y_pred = Lasso.predict(X_test)
rmse = sqrt(metrics.mean_squared_error(y_test, y_pred))

print("Train score:", Lasso.score(X_train, y_train))
print("Test score:", Lasso.score(X_test, y_test))
print("Number of coefficients used:", np.sum(Lasso.coef_!=0))
print("Root mean squared error:", rmse)
print(Lasso.coef_)
pd.Series(Lasso.coef_, index = X.columns)

#### Analysis

### MICE Imputation Dataset

In [ ]:
X = DatasetMICE.copy().select_dtypes([np.number]).drop(["12 month Cum Prod", "Year Drilled"], axis = 1)
scaler = StandardScaler()
sX = scaler.fit_transform(X)
y = DatasetMICE["12 month Cum Prod"]
sy = scaler.fit_transform(y.values.reshape((-1,1)))
alphas = 10**np.linspace(10,-2, 100)*0.05
coefs = []
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .4, random_state = 1)
Lasso = linear_model.Lasso(max_iter = 300000, tol = 0.0001, normalize = True)
for a in alphas:
    Lasso.set_params(alpha = a)
    Lasso.fit(X_train, y_train)
    coefs.append(Lasso.coef_)
ax = plt.gca()
ax.plot(alphas*2, coefs)
ax.set_xscale("log")
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('weights')
    
Lassocv = LassoCV(alphas = None, cv = 20, max_iter = 300000, normalize = True)
Lassocv.fit(X_train, y_train)

Lasso.set_params(alpha = Lassocv.alpha_)
Lasso.fit(X_train, y_train)

y_pred = Lasso.predict(X_test)
rmse = sqrt(metrics.mean_squared_error(y_test, y_pred))

print("Train score:", Lasso.score(X_train, y_train))
print("Test score:", Lasso.score(X_test, y_test))
print("Number of coefficients used:", np.sum(Lasso.coef_!=0))
print("Root mean squared error:", rmse)
print(Lasso.coef_)
pd.Series(Lasso.coef_, index = X.columns)

#### Analysis

### Seperating by Completion Type

In [ ]:

Category = "Completion Type"
DatasetII[Category] = OriginalDatasetwTypes[Category]
DatasetII["Sleeves"] = OriginalDatasetwTypes["Sleeves"]
DatasetII["P&P"] = OriginalDatasetwTypes["P&P"]
DatasetII["Frac Ports"] = OriginalDatasetwTypes["Frac Ports"]
DatasetII["Repeater Ports"] = OriginalDatasetwTypes["Repeater Ports"]
DatasetII["CT"] = OriginalDatasetwTypes["CT"]
DatasetII = DatasetII.dropna(axis=0, subset=['Completion Type'])

DatasetIISleeves = pd.concat([\
                               DatasetII[DatasetII[Category] == \
                                          "Sleeves"], \
                               DatasetII[DatasetII[Category] == \
                                           "Sleeves and P & P"]])
DatasetIISleeves = DatasetIISleeves.fillna(0)

DatasetIIPandP = pd.concat([\
                            DatasetII[DatasetII[Category] == \
                                      "P & P"], \
                            DatasetII[DatasetII[Category] == \
                                      "Sleeves and P & P"], \
                            DatasetII[DatasetII[Category] == \
                                      "Frac Ports and P & P"], \
                            DatasetII[DatasetII[Category] == \
                                       "P & P and CT"], \
                            DatasetII[DatasetII[Category] == \
                                      "Repeater Ports and P & P"]])
DatasetIIPandP = DatasetIIPandP.fillna(0)

DatasetIIFracPorts = pd.concat([\
                                DatasetII[DatasetII[Category] == \
                                         "Frac Ports"], \
                                DatasetII[DatasetII[Category] == \
                                         "Frac Ports and P & P"], \
                                DatasetII[DatasetII[Category] == \
                                         "Frac Ports and Repeater Ports"]])
DatasetIIFracPorts = DatasetIIFracPorts.fillna(0)

DatasetIIRepeaterPorts = pd.concat([\
                                    DatasetII[DatasetII[Category] == \
                                        "Frac Ports and Repeater Ports"],
                                    DatasetII[DatasetII[Category] == \
                                              "Repeater Ports and P & P"]])
DatasetIIRepeaterPorts = DatasetIIRepeaterPorts.fillna(0)

DatasetIICT = pd.concat([\
                         DatasetII[DatasetII[Category] == \
                                   "CT"], \
                         DatasetII[DatasetII[Category] == \
                                   "P & P and CT"]])
DatasetIICT = DatasetIICT.fillna(0)
DatasetIIOH = DatasetII[DatasetII[Category] == "OH"]
DatasetIIOH = DatasetIIOH.fillna(0)

DatasetIIScreen = DatasetII[DatasetII[Category] == "Screen"]
DatasetIIScreen = DatasetIIScreen.fillna(0)

DatasetIICementedL = DatasetII[DatasetII[Category] == "Cemented Liner"]
DatasetIICementedL = DatasetIICementedL.fillna(0)

DatasetIIPerforatedL = DatasetII[DatasetII[Category] == \
                                   "Perforated Liner"]
DatasetIIPerforatedL = DatasetIIPerforatedL.fillna(0)

DatasetIIother = pd.concat([DatasetIIOH, DatasetIIScreen, \
                           DatasetIICementedL, DatasetIIPerforatedL, DatasetIIFracPorts, DatasetIIRepeaterPorts])


### Sleeves

In [ ]:
X = DatasetIISleeves.copy().select_dtypes([np.number]).drop(["12 month Cum Prod", "Year Drilled", "Unnamed: 0"], axis = 1)
scaler = StandardScaler()
sX = scaler.fit_transform(X)
y = DatasetIISleeves["12 month Cum Prod"]
sy = scaler.fit_transform(y.values.reshape((-1,1)))
alphas = 10**np.linspace(10,-2, 100)*0.05
coefs = []
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .4, random_state = 1)
Lasso = linear_model.Lasso(max_iter = 300000, tol = 0.0001, normalize = True)
for a in alphas:
    Lasso.set_params(alpha = a)
    Lasso.fit(X_train, y_train)
    coefs.append(Lasso.coef_)
ax = plt.gca()
ax.plot(alphas*2, coefs)
ax.set_xscale("log")
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('weights')
    
Lassocv = LassoCV(alphas = None, cv = 20, max_iter = 300000, normalize = True)
Lassocv.fit(X_train, y_train)

Lasso.set_params(alpha = Lassocv.alpha_)
Lasso.fit(X_train, y_train)

y_pred = Lasso.predict(X_test)
rmse = sqrt(metrics.mean_squared_error(y_test, y_pred))

print("Train score:", Lasso.score(X_train, y_train))
print("Test score:", Lasso.score(X_test, y_test))
print("Number of coefficients used:", np.sum(Lasso.coef_!=0))
print("Root mean squared error:", rmse)
print(Lasso.coef_)
pd.Series(Lasso.coef_, index = X.columns)

#### Analysis

### P & P

In [ ]:
X = DatasetIIPandP.copy().select_dtypes([np.number]).drop(["12 month Cum Prod", "Year Drilled", "Unnamed: 0"], axis = 1)
scaler = StandardScaler()
sX = scaler.fit_transform(X)
y = DatasetIIPandP["12 month Cum Prod"]
sy = scaler.fit_transform(y.values.reshape((-1,1)))
alphas = 10**np.linspace(10,-2, 100)*0.05
coefs = []
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .4, random_state = 1)
Lasso = linear_model.Lasso(max_iter = 300000, tol = 0.0001, normalize = True)
for a in alphas:
    Lasso.set_params(alpha = a)
    Lasso.fit(X_train, y_train)
    coefs.append(Lasso.coef_)
ax = plt.gca()
ax.plot(alphas*2, coefs)
ax.set_xscale("log")
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('weights')
    
Lassocv = LassoCV(alphas = None, cv = 20, max_iter = 300000, normalize = True)
Lassocv.fit(X_train, y_train)

Lasso.set_params(alpha = Lassocv.alpha_)
Lasso.fit(X_train, y_train)

y_pred = Lasso.predict(X_test)
rmse = sqrt(metrics.mean_squared_error(y_test, y_pred))

print("Train score:", Lasso.score(X_train, y_train))
print("Test score:", Lasso.score(X_test, y_test))
print("Number of coefficients used:", np.sum(Lasso.coef_!=0))
print("Root mean squared error:", rmse)
print(Lasso.coef_)
pd.Series(Lasso.coef_, index = X.columns)

#### Analysis

### Other

In [ ]:
X = DatasetIIother.copy().select_dtypes([np.number]).drop(["12 month Cum Prod", "Year Drilled", "Unnamed: 0"], axis = 1)
scaler = StandardScaler()
sX = scaler.fit_transform(X)
y = DatasetIIother["12 month Cum Prod"]
sy = scaler.fit_transform(y.values.reshape((-1,1)))
alphas = 10**np.linspace(10,-2, 100)*0.05
coefs = []
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .4, random_state = 1)
Lasso = linear_model.Lasso(max_iter = 300000, tol = 0.0001, normalize = True)
for a in alphas:
    Lasso.set_params(alpha = a)
    Lasso.fit(X_train, y_train)
    coefs.append(Lasso.coef_)
ax = plt.gca()
ax.plot(alphas*2, coefs)
ax.set_xscale("log")
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('weights')
    
Lassocv = LassoCV(alphas = None, cv = 20, max_iter = 300000, normalize = True)
Lassocv.fit(X_train, y_train)

Lasso.set_params(alpha = Lassocv.alpha_)
Lasso.fit(X_train, y_train)

y_pred = Lasso.predict(X_test)
rmse = sqrt(metrics.mean_squared_error(y_test, y_pred))

print("Train score:", Lasso.score(X_train, y_train))
print("Test score:", Lasso.score(X_test, y_test))
print("Number of coefficients used:", np.sum(Lasso.coef_!=0))
print("Root mean squared error:", rmse)
print(Lasso.coef_)
pd.Series(Lasso.coef_, index = X.columns)

#### Analysis